In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

In [3]:
sentences = ["i like dog", "i love coffee", "i hate milk"]
word_list = " ".join(sentences).split()

vocab = list(set(word_list))
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for i, w in enumerate(vocab)}
n_class = len(vocab)

batch_size = 2
n_step = 2
n_hidden = 2


def make_data(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word2idx[n] for n in word[:-1]]
        target = word2idx[word[-1]]

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return input_batch, target_batch


input_batch, target_batch = make_data(sentences)
input_batch, target_batch = torch.Tensor(input_batch), torch.LongTensor(target_batch)
dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, batch_size, True)

C:\Users\boss\AppData\Local\Temp\ipykernel_5756\3093046672.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  input_batch, target_batch = torch.Tensor(input_batch), torch.LongTensor(target_batch)


In [4]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)
        # input_size 指的是每个单词用多少维的向量去编码
        # hidden_size 指的是输出的维度是多少
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, hidden, X):
        # X: [batch_size, n_step, n_class]
        X = X.transpose(0, 1)  # X : [n_step, batch_size, n_class]
        out, hidden = self.rnn(X, hidden)
        # out : [n_step, batch_size, num_directions(=1) * n_hidden]
        # hidden : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        out = out[-1]  # [batch_size, num_directions(=1) * n_hidden]
        model = self.fc(out)
        return model


model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
for epoch in range(5000):
    for x, y in loader:
        hidden = torch.zeros(1, x.shape[0], n_hidden)
        pred = model(hidden, x)
        loss = criterion(pred, y)

        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 1000 cost = 0.790806
Epoch: 1000 cost = 0.207991
Epoch: 2000 cost = 0.110075
Epoch: 2000 cost = 0.142711
Epoch: 3000 cost = 0.040230
Epoch: 3000 cost = 0.021265
Epoch: 4000 cost = 0.011444
Epoch: 4000 cost = 0.013657
Epoch: 5000 cost = 0.005312
Epoch: 5000 cost = 0.003269


In [15]:
input = [sen.split()[:2] for sen in sentences]
hidden = torch.zeros(1, len(input), n_hidden)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]

In [16]:
predict

tensor([[5],
        [6],
        [3]])

In [17]:
print([sen.split()[:2] for sen in sentences], '->', [idx2word[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
